In [ ]:
#Importing necessary packages
import pandas as pd
import numpy as np
import jpx_tokyo_market_prediction

In [ ]:
# Loading Train Set
path = "../input/jpx-tokyo-stock-exchange-prediction/"
usecols = ["Date","SecuritiesCode","Close"]
df = pd.read_csv(f"{path}train_files/stock_prices.csv", usecols=usecols)
df = df.sort_values(["SecuritiesCode","Date"], ascending=[True,True]).ffill().bfill()
df = df[df.Date>"2021-10-20"]
df = pd.pivot_table(df,values="Close",index="Date",columns="SecuritiesCode")

In [ ]:
def sharpe_ratio(d, f=0.01, h=100):
    r = np.mean(d,axis=0).fillna(0)
    s = d.std()
    v = s.fillna(s.mean()).values
    v = np.where(v>f,v*h,v)
    return (r/v).fillna(0).values

In [ ]:
%%time

env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

n = 15 # Best windows size in validation tests

for (prices, _, _, _, _, submission) in iter_test:
    pr = pd.pivot_table(prices,values="Close",index="Date",columns="SecuritiesCode")
    df = pd.concat([df, pr]).ffill().bfill()
    df_t = df.pct_change()       # Calculation of Returns from closings
    sr = sharpe_ratio(df_t[-n:]) # Sharpe Ratio in Rolling Window
    submission["Rank"] = len(submission)-np.argsort(np.argsort(np.array(sr)))-1
    env.predict(submission)